In [34]:
import arcpy
import os
import psycopg2
import pandas as pd
import numpy as np
from arcpy import sa


In [ ]:
#Create database connection - only do this step if connection in arcgis pro not already established
arcpy.management.CreateDatabaseConnection(
    out_folder_path=r"C:\Users\KOlso\Documents\GIS5572",
    out_name="Test",
    database_platform="POSTGRESQL",
    instance="34.30.71.239",
    account_authentication="DATABASE_AUTH",
    username="postgres",
    password='',
    save_user_pass="SAVE_USERNAME",
    database="gis_data",
    schema="",
    version_type="TRANSACTIONAL",
    version="",
    date=None,
    auth_type="",
    project_id="",
    default_dataset="",
    refresh_token='',
    key_file=None,
    role="",
    warehouse="",
    advanced_options=""
)

## Connect to databse - retrieve temperature data for Aug 31, 2024 - convert to fc

In [49]:
#connect to db and grab the columns we want to use, turn this into a fc we'll use as basis for interpolations.
#In this case we are using August 31st data but could in theory pick any day from August since that is what we have in db
conn = psycopg2.connect(
    dbname="gis_data",
    user="",
    password="",
    host="34.30.71.239",
    port="5432"
)

cur = conn.cursor()
cur.execute("""
    SELECT stid, "20240831", name, ST_AsText(geometry) 
    FROM gis_data.public.mesonet
""")
rows = cur.fetchall()
conn.close()

#Prepare new feature class - "alt" is used here because I considered another method I abandoned and I didn't want to change all the variables
output_fc = r"G:\ArcGIS\Projects\5572\5572.gdb\mesonet_fc_alt"
sr = arcpy.SpatialReference(4326)  # Adjust if your data uses a different SRID

# Create the feature class
arcpy.management.CreateFeatureclass(
    out_path=r"G:\ArcGIS\Projects\5572\5572.gdb",
    out_name="mesonet_fc_alt",
    geometry_type="POINT",
    spatial_reference=sr
)

# Add fields to the feature class
arcpy.management.AddField(output_fc, "stid", "TEXT", field_length=50)
arcpy.management.AddField(output_fc, "temp_20240831", "DOUBLE")
arcpy.management.AddField(output_fc, "name", "TEXT", field_length=100)

# Insert features from the db table
with arcpy.da.InsertCursor(output_fc, ["stid", "temp_20240831", "name", "SHAPE@XY"]) as cursor:
    for stid, temp, name, geom_wkt in rows:
        try:
            # Convert WKT to geometry using arcpy
            geom = arcpy.FromWKT(geom_wkt)  # Convert WKT to geometry
            cursor.insertRow((stid, temp, name, (geom.centroid.X, geom.centroid.Y)))  # Use centroid for point
        except Exception as e:
            print(f"Error processing WKT for {stid}: {e}")

print("Feature class created from psycopg2 data.")

✅ Feature class created from psycopg2 data.


## Interpolate with all points

In [16]:
#inerpolation 1 IDW
with arcpy.EnvManager(scratchWorkspace=r"G:\ArcGIS\Projects\5572\5572.gdb"):
    out_raster = arcpy.sa.Idw(
        in_point_features="mesonet_fc_alt",
        z_field="temp_20240831",
        cell_size=0.0005,
        power=2,
        search_radius="VARIABLE 12",
        in_barrier_polyline_features=None
    )
    out_raster.save(r"G:\ArcGIS\Projects\5572\5572.gdb\mesonet_alt_IDW_all")




In [17]:
#interpolation 2 - Ordinary Kriging from spatial analyst toolbox

with arcpy.EnvManager(scratchWorkspace=r"G:\ArcGIS\Projects\5572\5572.gdb"):
    out_surface_raster = arcpy.sa.Kriging(
        in_point_features="mesonet_fc_alt",
        z_field="temp_20240831",
        kriging_model="Spherical # # # #",
        cell_size=0.0005,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    out_surface_raster.save(r"G:\ArcGIS\Projects\5572\5572.gdb\mesonet_alt_okri_all")


In [18]:
#interpolation #3 - Universal Kriging from spatial analyst toolbox

with arcpy.EnvManager(scratchWorkspace=r"G:\ArcGIS\Projects\5572\5572.gdb"):
    out_surface_raster = arcpy.sa.Kriging(
        in_point_features="mesonet_fc_alt",
        z_field="temp_20240831",
        kriging_model="LinearDrift 0.000500 # # #",
        cell_size=0.0005,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    out_surface_raster.save(r"G:\ArcGIS\Projects\5572\5572.gdb\mesonet_alt_ukri_all")

In [1]:
#accuracy assessment - one approach similar to what Nicole did for Elevation but I wanted to do it differently but this is still here
# arcpy.ga.ExploratoryInterpolation(
#     in_features="mesonet_fc_alt",
#     value_field="temp_20240831",
#     out_cv_table=r"G:\ArcGIS\Projects\5572\5572.gdb\Temp_0831_ExpInt_table",
#     out_geostat_layer="test_output_ex_int",
#     interp_methods="ORDINARY_KRIGING;UNIVERSAL_KRIGING;IDW",
#     comparison_method="SINGLE",
#     criterion="ACCURACY",
#     criteria_hierarchy="ACCURACY PERCENT #",
#     weighted_criteria="ACCURACY 1",
#     exclusion_criteria=None
# )

<Result 'G:\\ArcGIS\\Projects\\5572\\5572.gdb\\Temp_0831_ExpInt_table'>

## Validate results - Using K-folds cross validation - 3 points per fold

In [30]:
# Set environment - probably not necessary since I'm already working here
arcpy.env.workspace = r"G:\ArcGIS\Projects\5572\5572.gdb"
arcpy.env.overwriteOutput = True

# Inputs used for all three cross validations we'll perform. Cell size is partially arbitrary but at this latitude is roughly 50 square meters
input_fc = "mesonet_fc_alt"
z_field = "temp_20240831"
id_field = "stid"
cell_size = .0005  # Adjustable but should be in this scale since data is all 4326 so unit is decimal degrees
num_folds = 9
fold_size = 3
output_table_base = "cv_results"  # Base name for all output tables

# Set extent to match the feature class - this is necessary so extent is same for all folds (otherwise edge points may get left out sometimes)
extent = arcpy.Describe(input_fc).extent
arcpy.env.extent = extent

# Create list of all OIDs and assign to folds - these are the folds used in the k-folds analysis that will be used for each cross validation
oids = [row[0] for row in arcpy.da.SearchCursor(input_fc, ["OID@"])]
random.shuffle(oids)
folds = [oids[i:i+fold_size] for i in range(0, len(oids), fold_size)]

# Create a function that will create a table for each cross validation of an interpolation method
def create_validation_table(output_table_name):
    """Creates a validation table with MAE and difference columns"""
    if arcpy.Exists(output_table_name):
        arcpy.management.Delete(output_table_name)
    arcpy.management.CreateTable(arcpy.env.workspace, output_table_name)
    arcpy.management.AddField(output_table_name, "Fold", "SHORT")
    arcpy.management.AddField(output_table_name, "OID", "LONG")
    arcpy.management.AddField(output_table_name, "stid", "TEXT", field_length=50)
    arcpy.management.AddField(output_table_name, "Actual", "DOUBLE")
    arcpy.management.AddField(output_table_name, "Predicted", "DOUBLE")
    arcpy.management.AddField(output_table_name, "Difference", "DOUBLE")

### IDW cross validation

In [31]:
# Set the output table name for IDW results
output_table_idw = f"{output_table_base}_IDW"

# Create a table to store results for IDW
create_validation_table(output_table_idw)

# Perform IDW cross-validation
for i, test_oids in enumerate(folds):
    print(f"Processing fold {i+1}...")

    # Create training and testing feature layers
    train_layer = f"train_{i}"
    test_layer = f"test_{i}"
    test_oid_str = ",".join(str(oid) for oid in test_oids)
    train_where = f"OBJECTID NOT IN ({test_oid_str})"
    test_where = f"OBJECTID IN ({test_oid_str})"

    arcpy.management.MakeFeatureLayer(input_fc, train_layer, train_where)
    arcpy.management.MakeFeatureLayer(input_fc, test_layer, test_where)

    # Run IDW interpolation
    idw_raster = arcpy.sa.Idw(
        in_point_features=train_layer,
        z_field=z_field,
        cell_size=cell_size
    )

    idw_raster_name = f"idw_fold_{i+1}"
    idw_raster.save(idw_raster_name)

    # Extract predicted values to test points 
    test_with_prediction = f"test_with_pred_{i}"
    arcpy.sa.ExtractValuesToPoints(test_layer, idw_raster, test_with_prediction, interpolate_values="NONE")

    # Record actual vs predicted temperatures at each point
    fields = ["Fold", "OID@", id_field, z_field, "RASTERVALU"]
    with arcpy.da.SearchCursor(test_with_prediction, ["OID@", id_field, z_field, "RASTERVALU"]) as cursor_in, \
         arcpy.da.InsertCursor(output_table_idw, ["Fold", "OID", "stid", "Actual", "Predicted", "Difference"]) as cursor_out:
        for row in cursor_in:
            actual_value = row[2]
            predicted_value = row[3]
            
            # Check if both actual and predicted values are not None
            if actual_value is not None and predicted_value is not None:
                diff = predicted_value - actual_value  # Difference = Predicted - Actual
                cursor_out.insertRow((i + 1, row[0], row[1], actual_value, predicted_value, diff))
            else:
                # If either value is None, you can handle it by skipping or inserting a null value
                cursor_out.insertRow((i + 1, row[0], row[1], actual_value, predicted_value, None))


    # Clean up
    arcpy.management.Delete(train_layer)
    arcpy.management.Delete(test_layer)
    arcpy.management.Delete(test_with_prediction)
    arcpy.management.Delete(idw_raster)

print("Cross-validation with IDW complete. Results saved to 'cv_results_idw'.")

Processing fold 1...
Processing fold 2...
Processing fold 3...
Processing fold 4...
Processing fold 5...
Processing fold 6...
Processing fold 7...
Processing fold 8...
Processing fold 9...
✅ Cross-validation with IDW complete. Results saved to 'idw_cv_results'.


### Ordinary Kriging Cross Validation

In [32]:
# Set the output table name for OK results
output_table_ok = f"{output_table_base}_OK"

# Create a table to store results for Ordinary Kriging
create_validation_table(output_table_ok)

# Perform Ordinary Kriging cross-validation
for i, test_oids in enumerate(folds):
    print(f"Processing fold {i+1}...")

    # Create training and testing feature layers
    train_layer = f"train_{i}"
    test_layer = f"test_{i}"
    test_oid_str = ",".join(str(oid) for oid in test_oids)
    train_where = f"OBJECTID NOT IN ({test_oid_str})"
    test_where = f"OBJECTID IN ({test_oid_str})"

    arcpy.management.MakeFeatureLayer(input_fc, train_layer, train_where)
    arcpy.management.MakeFeatureLayer(input_fc, test_layer, test_where)

    # Run Ordinary Kriging interpolation
    ok_result = arcpy.sa.Kriging(
        in_point_features=train_layer,
        z_field=z_field,
        kriging_model="Spherical",  # Change as needed
        cell_size=cell_size,
        search_radius="VARIABLE 12"
    )

    ok_raster_name = f"ok_fold_{i+1}"
    ok_result.save(ok_raster_name)

    # Extract predicted values to test points
    test_with_prediction = f"test_with_pred_{i}"
    arcpy.sa.ExtractValuesToPoints(test_layer, ok_result, test_with_prediction, interpolate_values="NONE")

    # Record actual vs predicted temperatures at each point
    fields = ["Fold", "OID@", id_field, z_field, "RASTERVALU"]
    with arcpy.da.SearchCursor(test_with_prediction, ["OID@", id_field, z_field, "RASTERVALU"]) as cursor_in, \
         arcpy.da.InsertCursor(output_table_ok, ["Fold", "OID", "stid", "Actual", "Predicted", "Difference"]) as cursor_out:
        for row in cursor_in:
            actual_value = row[2]
            predicted_value = row[3]
            
            # Check if both actual and predicted values are not None
            if actual_value is not None and predicted_value is not None:
                diff = predicted_value - actual_value  # Difference = Predicted - Actual
                cursor_out.insertRow((i + 1, row[0], row[1], actual_value, predicted_value, diff))
            else:
                # If either value is None, you can handle it by skipping or inserting a null value
                cursor_out.insertRow((i + 1, row[0], row[1], actual_value, predicted_value, None))

    # Clean up
    arcpy.management.Delete(train_layer)
    arcpy.management.Delete(test_layer)
    arcpy.management.Delete(test_with_prediction)
    arcpy.management.Delete(ok_result)

print("Cross-validation with Ordinary Kriging complete. Results saved to 'cv_results_OK'.")

Processing fold 1...
Processing fold 2...
Processing fold 3...
Processing fold 4...
Processing fold 5...
Processing fold 6...
Processing fold 7...
Processing fold 8...
Processing fold 9...
Cross-validation with Ordinary Kriging complete. Results saved to 'cv_results_OK'.


### Universal Kriging Cross Validation

In [33]:
# Set the output table name for UK results
output_table_uk = f"{output_table_base}_UK"

# Create a table to store results for Universal Kriging
create_validation_table(output_table_uk)

# Perform Universal Kriging cross-validation
for i, test_oids in enumerate(folds):
    print(f"Processing fold {i+1}...")

    # Create training and testing feature layers
    train_layer = f"train_{i}"
    test_layer = f"test_{i}"
    test_oid_str = ",".join(str(oid) for oid in test_oids)
    train_where = f"OBJECTID NOT IN ({test_oid_str})"
    test_where = f"OBJECTID IN ({test_oid_str})"

    arcpy.management.MakeFeatureLayer(input_fc, train_layer, train_where)
    arcpy.management.MakeFeatureLayer(input_fc, test_layer, test_where)

    # Record actual vs predicted temperatures at each point
    uk_result = arcpy.sa.Kriging(
        in_point_features=train_layer,
        z_field=z_field,
        kriging_model="LinearDrift 0.000500 # # #",
        cell_size=cell_size,
        search_radius="VARIABLE 12",
    )

    uk_raster_name = f"uk_fold_{i+1}"
    uk_result.save(uk_raster_name)

    # Extract predicted values to test points
    test_with_prediction = f"test_with_pred_{i}"
    arcpy.sa.ExtractValuesToPoints(test_layer, uk_result, test_with_prediction, interpolate_values="NONE")

    # Record actual vs predicted
    fields = ["Fold", "OID@", id_field, z_field, "RASTERVALU"]
    with arcpy.da.SearchCursor(test_with_prediction, ["OID@", id_field, z_field, "RASTERVALU"]) as cursor_in, \
         arcpy.da.InsertCursor(output_table_uk, ["Fold", "OID", "stid", "Actual", "Predicted", "Difference"]) as cursor_out:
        for row in cursor_in:
            actual_value = row[2]
            predicted_value = row[3]
            
            # Check if both actual and predicted values are not None
            if actual_value is not None and predicted_value is not None:
                diff = predicted_value - actual_value  # Difference = Predicted - Actual
                cursor_out.insertRow((i + 1, row[0], row[1], actual_value, predicted_value, diff))
            else:
                # If either value is None, you can handle it by skipping or inserting a null value
                cursor_out.insertRow((i + 1, row[0], row[1], actual_value, predicted_value, None))
    # Clean up
    arcpy.management.Delete(train_layer)
    arcpy.management.Delete(test_layer)
    arcpy.management.Delete(test_with_prediction)
    arcpy.management.Delete(ok_result)
   
print("Cross-validation with Universal Kriging complete. Results saved to 'cv_results_UK'.")

Processing fold 1...
Processing fold 2...
Processing fold 3...
Processing fold 4...
Processing fold 5...
Processing fold 6...
Processing fold 7...
Processing fold 8...
Processing fold 9...
Cross-validation with Universal Kriging complete. Results saved to 'cv_results_UK'.


### Calculate RMSE and MAE for each interpolation Type

In [38]:
# Create dictionary for the table names that we'll use for calulating our evaluation metrics
table_paths = {
    "IDW": "cv_results_IDW",
    "OrdinaryKriging": "cv_results_OK",
    "UniversalKriging": "cv_results_UK"
}

# create list for results to sit in
results = []

for method, table in table_paths.items():
    # Load into pandas
    arr = arcpy.da.TableToNumPyArray(table, ["Actual", "Predicted"])
    df = pd.DataFrame(arr)

    # Drop nulls
    df = df.dropna()

    # Extract values
    y_true = df["Actual"].to_numpy()
    y_pred = df["Predicted"].to_numpy()

    # Calculate MAE
    mae = np.mean(np.abs(y_pred - y_true))

    # Calculate RMSE
    rmse = np.sqrt(np.mean((y_pred - y_true) ** 2))

    results.append({
        "Method": method,
        "MAE": round(mae, 3),
        "RMSE": round(rmse, 3)
    })

# Create and display results table
results_df = pd.DataFrame(results)
print(results_df)

             Method    MAE   RMSE
0               IDW  0.431  0.564
1   OrdinaryKriging  0.510  0.589
2  UniversalKriging  0.469  0.655


### Convert best interpolation to points, join table with differences to a feature class for upload to db

In [39]:
#Looks like IDW has the smallest MAE and RMSE - we're only ever off by about half a degree or less. 

#Now we convert our IDW Raster from earlier to point for easier display in the webmap via database connection
arcpy.conversion.RasterToPoint(
    in_raster="Mesonet_alt_IDW",
    out_point_features=r"G:\ArcGIS\Projects\5572\5572.gdb\Mesonet_IDW_RtP",
    raster_field="Value"
)


<Result 'G:\\ArcGIS\\Projects\\5572\\5572.gdb\\Mesonet_IDW_RtP'>

In [40]:
#set variables to join tables so we can have  both actual and predicted results

source_table = "cv_results_IDW"
target_fc = "mesonet_fc_alt"
join_field = "stid"
fields_to_add = ["Predicted", "Difference"]

# Create dictionary to store data from cv_results_IDW table
value_dict = {}
with arcpy.da.SearchCursor(source_table, [join_field] + fields_to_add) as cursor:
    for row in cursor:
        value_dict[row[0]] = row[1:]  # key: stid, value: (Predicted, Difference)

#Add new fields to my feature class
arcpy.management.AddField(target_fc, "IDW_Predicted", "DOUBLE")
arcpy.management.AddField(target_fc, "Difference", "DOUBLE")

#Update feature class with values from dictionary, joining on stid field
with arcpy.da.UpdateCursor(target_fc, [join_field, "IDW_Predicted", "Difference"]) as cursor:
    for row in cursor:
        stid = row[0]
        if stid in value_dict:
            row[1], row[2] = value_dict[stid]
            cursor.updateRow(row)

print("Fields added and populated successfully.")

Fields added and populated successfully.


### Upload interpolatation layer and feature class with differences to db

In [42]:
#First we'll do the newly updated feature class with the difference column (requirement 2F from lab instructions)

# set feature class variable
fc1 = "mesonet_fc_alt"

# Connect to PostGIS
conn = psycopg2.connect(
    dbname="gis_data",
    user="",
    password="",
    host="34.30.71.239",
    port="5432"
)
cur = conn.cursor()

# Drop and recreate table
cur.execute("DROP TABLE IF EXISTS temp_station_points;")
cur.execute("""
    CREATE TABLE temp_station_points (
        id SERIAL PRIMARY KEY,
        stid TEXT,
        temp_20240831 DOUBLE PRECISION,
        name TEXT,
        idw_predicted DOUBLE PRECISION,
        difference DOUBLE PRECISION,
        geom GEOMETRY(POINT, 4326)
    );
""")
conn.commit()

# Insert rows from feature class
with arcpy.da.SearchCursor(fc1, ["SHAPE@XY", "stid", "temp_20240831", "name", "IDW_Predicted", "Difference"]) as cursor:
    for row in cursor:
        (x, y) = row[0]
        stid = row[1]
        temp = row[2]
        name = row[3]
        predicted = row[4]
        difference = row[5]
        cur.execute("""
            INSERT INTO temp_station_points (stid, temp_20240831, name, idw_predicted, difference, geom)
            VALUES (%s, %s, %s, %s, %s, ST_SetSRID(ST_Point(%s, %s), 4326));
        """, (stid, temp, name, predicted, difference, x, y))

conn.commit()
cur.close()
conn.close()

print("Temperature points uploaded to db")

In [44]:
#Next we'll push the points converted from the raster

# set feature class variable
fc2 = "Mesonet_IDW_RtP"

# Connect to PostGIS
conn = psycopg2.connect(
    dbname="gis_data",
    user="",
    password="",
    host="34.30.71.239",
    port="5432"
)
cur = conn.cursor()

# Recreate the table
cur.execute("DROP TABLE IF EXISTS temp_raster_points_IDW;")
cur.execute("""
    CREATE TABLE temp_raster_points_IDW (
        id SERIAL PRIMARY KEY,
        pointid INTEGER,
        grid_code DOUBLE PRECISION,
        geom GEOMETRY(POINT, 4326)
    );
""")
conn.commit()

# Insert rows from the feature class
with arcpy.da.SearchCursor(fc2, ["SHAPE@XY", "pointid", "grid_code"]) as cursor:
    for (shape_xy, pointid, grid_code) in cursor:
        x, y = shape_xy
        cur.execute("""
            INSERT INTO temp_raster_points_IDW (pointid, grid_code, geom)
            VALUES (%s, %s, ST_SetSRID(ST_Point(%s, %s), 4326));
        """, (pointid, grid_code, x, y))

conn.commit()
cur.close()
conn.close()

print("Temperature raster ponts uploaded to db")

In [46]:
#lastly, we'll send the accuracy assessment table from ealier

#This one is just a dataframe, not a feature class, so a slightly different approach is needed.

import psycopg2

# Connection setup
conn = psycopg2.connect(
    dbname="gis_data",
    user="",
    password="",
    host="34.30.71.239",
    port="5432"
)
cur = conn.cursor()

# Create table
cur.execute("""
    DROP TABLE IF EXISTS interpolation_stats;
    CREATE TABLE interpolation_stats (
        method TEXT,
        mae DOUBLE PRECISION,
        rmse DOUBLE PRECISION
    );
""")

# Insert rows
for _, row in results_df.iterrows():
    cur.execute("""
        INSERT INTO interpolation_stats (method, mae, rmse)
        VALUES (%s, %s, %s);
    """, (row["Method"], row["MAE"], row["RMSE"]))

conn.commit()
cur.close()
conn.close()

print("Accucary assessment table uploaded to db")

Accucary assessment table uploaded to db
